In [2]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install rdkit


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error


c:\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Define a Reaction dataset class
class ReactionDataset(Dataset):
    def __init__(self, X, y_G_act, y_G_r):
        self.X = X
        self.y_G_act = y_G_act
        self.y_G_r = y_G_r

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y_G_act[idx], self.y_G_r[idx]

In [5]:
# Read the dataset
df = pd.read_csv('full_dataset.csv') 

# Prepare the feature matrix X and target variables y
X_smiles = df['rxn_smiles']
y_G_act = df['G_act']
y_G_r = df['G_r']

In [6]:
# Define the encoding function
def encode_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
    bitstring = fp.ToBitString()
    features = np.array([int(bit) for bit in bitstring], dtype=np.float32)
    features_tensor = torch.tensor(features)
    return features_tensor

# Encode SMILES strings to fingerprints
X_fingerprints = []

for idx, smiles in enumerate(X_smiles):
    reactants, products = smiles.split('>>')
    reactant_fingerprints = [encode_smiles(reactant) for reactant in reactants.split('.')]
    product_fingerprints = [encode_smiles(product) for product in products.split('.')]
    X_fingerprints.append(reactant_fingerprints + product_fingerprints)

# Convert the list of fingerprints and target values to tensors
X_tensor = torch.stack([torch.cat(fingerprints) for fingerprints in X_fingerprints])

# Print the fingerprint tensor
print(X_tensor)
print(X_tensor.shape)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([5269, 3072])


In [7]:
# Convert the data into PyTorch tensors
y_G_act_tensor = torch.tensor(y_G_act.values, dtype=torch.float32)
y_G_r_tensor = torch.tensor(y_G_r.values, dtype=torch.float32)

In [8]:
# Split the dataset into training and testing sets
X_train, X_test, y_G_act_train, y_G_act_test, y_G_r_train, y_G_r_test = train_test_split(X_tensor, y_G_act_tensor, y_G_r_tensor, test_size=0.2, random_state=42)


In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(64, 64)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)
        self.fc3_act = nn.Linear(64, 1)  # Output layer for G_act
        self.fc3_r = nn.Linear(64, 1)  # Output layer for G_r

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        output_act = self.fc3_act(x)  # Output for G_act
        output_r = self.fc3_r(x)  # Output for G_r
        return output_act, output_r

In [10]:
# Instantiate the model
model = NeuralNetwork(X_train.shape[1])

In [11]:
# Define the loss function and optimizer
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
# Define the data loaders
train_dataset = ReactionDataset(X_train, y_G_act_train, y_G_r_train)
test_dataset = ReactionDataset(X_test, y_G_act_test, y_G_r_test)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32)

In [13]:
# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, targets_G_act, targets_G_r in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs_G_act, outputs_G_r = outputs  # Separate the outputs into two tensors
        loss_G_act = criterion(outputs_G_act, targets_G_act.unsqueeze(1))
        loss_G_r = criterion(outputs_G_r, targets_G_r.unsqueeze(1))
        loss = loss_G_act + loss_G_r
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")

Epoch 1/1000, Loss: 30.45531959244699
Epoch 2/1000, Loss: 19.1097511956186
Epoch 3/1000, Loss: 13.007705002120048
Epoch 4/1000, Loss: 11.293037862488717
Epoch 5/1000, Loss: 10.222045212080985
Epoch 6/1000, Loss: 9.683159777612397
Epoch 7/1000, Loss: 9.374874750773111
Epoch 8/1000, Loss: 8.91071718750578
Epoch 9/1000, Loss: 8.63733368208914
Epoch 10/1000, Loss: 8.315470919464573
Epoch 11/1000, Loss: 8.135114785396691
Epoch 12/1000, Loss: 7.986842505859606
Epoch 13/1000, Loss: 7.905284968289462
Epoch 14/1000, Loss: 7.672869888218966
Epoch 15/1000, Loss: 7.726050250457995
Epoch 16/1000, Loss: 7.437041835351423
Epoch 17/1000, Loss: 7.472227949084657
Epoch 18/1000, Loss: 7.313329494360722
Epoch 19/1000, Loss: 7.2551981066212505
Epoch 20/1000, Loss: 7.212780616500161
Epoch 21/1000, Loss: 7.133050362269084
Epoch 22/1000, Loss: 7.052901990485914
Epoch 23/1000, Loss: 6.877286119894548
Epoch 24/1000, Loss: 6.751619125857498
Epoch 25/1000, Loss: 6.884031884598009
Epoch 26/1000, Loss: 6.7937338605

In [14]:
# Evaluate the model on the testing data
model.eval()
with torch.no_grad():
    test_loss_G_act = 0.0
    test_loss_G_r = 0.0
    total_mse = 0.0
    total_mae = 0.0

    for inputs, targets_G_act, targets_G_r in test_dataloader:
        outputs = model(inputs)
        outputs_G_act, outputs_G_r = outputs  # Separate the outputs into two tensors

        loss_G_act = criterion(outputs_G_act, targets_G_act.unsqueeze(1))
        loss_G_r = criterion(outputs_G_r, targets_G_r.unsqueeze(1))

        test_loss_G_act += loss_G_act.item()
        test_loss_G_r += loss_G_r.item()

        mse_G_act = mean_squared_error(targets_G_act.cpu(), outputs_G_act.cpu().detach())
        mae_G_act = mean_absolute_error(targets_G_act.cpu(), outputs_G_act.cpu().detach())
        mse_G_r = mean_squared_error(targets_G_r.cpu(), outputs_G_r.cpu().detach())
        mae_G_r = mean_absolute_error(targets_G_r.cpu(), outputs_G_r.cpu().detach())

        total_mse += mse_G_act.item() + mse_G_r.item()
        total_mae += mae_G_act.item() + mae_G_r.item()

    average_test_loss_G_act = test_loss_G_act / len(test_dataloader)
    average_test_loss_G_r = test_loss_G_r / len(test_dataloader)

    avg_mse = total_mse / len(test_dataloader)
    avg_mae = total_mae / len(test_dataloader)

    print(f"Average Test Loss G_act: {average_test_loss_G_act}")
    print(f"Average Test Loss G_r: {average_test_loss_G_r}")
    
    print(f"Mean Squared Error (MSE): {avg_mse:.4f}")
    print(f"Mean Absolute Error (MAE): {avg_mae:.4f}")

Average Test Loss G_act: 3.3577062072175923
Average Test Loss G_r: 4.0672737468372695
Mean Squared Error (MSE): 64.6612
Mean Absolute Error (MAE): 8.3677


# For showing how it works, we will use the following example:

In [15]:

# Read the dataset
df = pd.read_csv('debuging_dataset.csv')

# Extract the SMILES strings
smiles = df['rxn_smiles']

print(f"G_act {df['G_act']} G_r {df['G_r']}  ")

# Encode SMILES strings to fingerprints
X_fingerprints = []

for idx, smiles in enumerate(smiles):
    reactants, products = smiles.split('>>')
    reactant_fingerprints = [encode_smiles(reactant) for reactant in reactants.split('.')]
    product_fingerprints = [encode_smiles(product) for product in products.split('.')]
    X_fingerprints.append(reactant_fingerprints + product_fingerprints)

# Convert the list of fingerprints and target values to tensors
X_test = torch.stack([torch.cat(fingerprints) for fingerprints in X_fingerprints])


G_act 0     15.875896
1     15.155477
2     18.013824
3     23.687079
4     23.438094
5     20.969801
6     11.722625
7      7.268044
8      6.289673
9     16.044475
10    13.604005
Name: G_act, dtype: float64 G_r 0    -51.881526
1    -51.398681
2    -66.822349
3    -62.481289
4    -84.836459
5    -57.785046
6    -63.721331
7    -86.996102
8    -86.799671
9    -50.409197
10   -51.008126
Name: G_r, dtype: float64  


In [16]:

# Set the model to evaluation mode
model.eval()

# Make predictions on test inputs
with torch.no_grad():
    test_inputs = X_test  
    predictions_G_act, predictions_G_r = model(test_inputs)  # Separate the predictions into two tensors

# Convert predictions to numpy arrays
predictions_G_act = predictions_G_act.numpy()
predictions_G_r = predictions_G_r.numpy()


print("Predictions for G_act:")
for pred in predictions_G_act:
    print(pred[0])

print("Predictions for G_r:")
for pred in predictions_G_r:
    print(pred[0])

Predictions for G_act:
15.393006
15.140726
18.152287
23.367092
23.329334
20.518341
12.146641
6.154443
6.6333065
16.150503
13.550837
Predictions for G_r:
-51.71132
-50.58262
-65.53673
-62.80134
-83.415535
-56.07112
-63.67861
-87.32187
-78.072044
-49.272835
-50.920586
